In [11]:
from moexalgo import Market, Ticker
from datetime import datetime, timedelta
import pandas as pd
import time

In [12]:
stocks = Market('stocks')

In [13]:
# Задаем параметры для загрузки данных
end_date = datetime.now()
start_date = end_date - timedelta(days=10)  # Пример: данные за последние 6 месяцев

# Создаем объект рынка
stocks_market = Market('stocks')

# Создаем пустой DataFrame для хранения данных
all_tradestats = pd.DataFrame()

# Итерируемся по датам и загружаем данные для каждой даты
current_date = start_date
while current_date <= end_date:
    # Получаем данные по всем акциям за текущую дату
    tradestats_date = stocks_market.tradestats(date=current_date)
    
    # Конкатенируем полученные данные с общим DataFrame
    all_tradestats = pd.concat([all_tradestats, tradestats_date])
    
    # Переходим к следующей дате
    current_date += timedelta(days=1)

# Группируем данные по тикеру и считаем средний объем торгов за период
average_volume = all_tradestats.groupby('ticker')['vol'].mean()

# Сортируем по убыванию среднего объема и берем топ-5
top_5_liquid_tickers = average_volume.sort_values(ascending=False).head(5).index

# Выводим топ-5 тикеров
print(top_5_liquid_tickers)

Index(['MTLR', 'SGZH', 'VTBR', 'RNFT', 'SBER'], dtype='object', name='ticker')


In [14]:
# Задаем параметры для загрузки данных
end_date = datetime.now()
start_date = end_date - timedelta(days=7)  # Данные за последнюю неделю

# Создаем пустой DataFrame для хранения данных
all_tradestats = pd.DataFrame()

# Итерируемся по каждому тикеру
for ticker in top_5_liquid_tickers:
    # Создаем объект тикера
    current_ticker = Ticker(ticker)
    
    # Создаем пустой DataFrame для хранения данных по текущему тикеру
    ticker_tradestats = pd.DataFrame()
    
    # Итерируемся по датам и загружаем данные для каждой даты
    current_date = start_date
    while current_date <= end_date:
        # Получаем данные по текущему тикеру за текущую дату
        tradestats_date = current_ticker.tradestats(date=current_date)
        
        # Конкатенируем полученные данные с общим DataFrame по текущему тикеру
        ticker_tradestats = pd.concat([ticker_tradestats, tradestats_date])
        
        # Переходим к следующей дате
        current_date += timedelta(days=1)
    
    # Добавляем данные по текущему тикеру в общий DataFrame
    all_tradestats = pd.concat([all_tradestats, ticker_tradestats])

    # Задержка перед следующим запросом
    time.sleep(0.5)

    # Сохраняем данные по текущему тикеру в CSV файл
    ticker_tradestats.to_csv(f'tradestats_{ticker}.csv', index=None)